In [1]:
from astropy.io import fits
import astropy
import numpy as np
import matplotlib.pyplot as plt
from jupyterthemes import jtplot

jtplot.style()

In [2]:
hdul_3fgl = fits.open('data/gll_psc_v16.fit')
hdul_4fgl = fits.open('data/gll_psc_v27.fit')

In [3]:
# The classes
classes = hdul_3fgl[1].data['CLASS1']
agn_classes = ['agn  ', 'FSRQ ', 'fsrq ', 'BLL  ', 'bll  ', 'BCU  ', 'bcu  ', 'RDG  ', 'rdg  ', 'NLSY1', 'nlsy1', 'ssrq ', 'sey  ']
pulsar_classes = ['PSR  ', 'psr  ']
no_class = '     '
agn_mask = np.isin(classes, agn_classes)
pulsar_mask = np.isin(classes, pulsar_classes)
noclass_mask = classes == no_class
bad_data_mask = hdul_3fgl[1].data['Signif_Curve'] == 0.0 # Causes ln(0.0)

# The data, sorted by class
pulsars = hdul_3fgl[1].data[pulsar_mask & ~bad_data_mask]
agns = hdul_3fgl[1].data[agn_mask & ~bad_data_mask]
data = hdul_3fgl[1].data[(agn_mask | pulsar_mask) & ~bad_data_mask] # data is both pulsars and AGNs

In [81]:
# The 11 features used for the 3FGL catalog

# The easy ones
glat = data['GLAT']
glon = data['GLON']
ln_energy_flux100 = np.log(data['Energy_Flux100'])
ln_unc_energy_flux100 = np.log(data['Unc_Energy_Flux100'])
ln_signif_curve = np.log(data['Signif_Curve'])
ln_var_index = np.log(data['Variability_Index'])

# Hardness ratios
ef1 = data['Flux100_300']
ef2 = data['Flux300_1000']
ef3 = data['Flux1000_3000']
ef4 = data['Flux3000_10000']
ef5 = data['Flux10000_100000']
hr12 = (ef2 - ef1) / (ef2 + ef1)
hr23 = (ef3 - ef2) / (ef3 + ef2)
hr34 = (ef4 - ef3) / (ef4 + ef3)
hr45 = (ef5 - ef4) / (ef5 + ef4)

# 500 MeV index
alpha = data['Spectral_Index']
beta = data['beta']
gamma = data['Spectral_Index']
b = data['Exp_Index']
E_c = data['Cutoff'] # In MeV
E_0 = data['Pivot_Energy'] # In MeV
mev_500_index = np.zeros(data.shape)
for i, point in enumerate(data):
    if point['SpectrumType'] in ['PowerLaw', 'PLExpCutoff', 'PLSuperExpCutoff']:
        if b[i] == float('-inf'):
            b[i] = 1
        mev_500_index[i] = gamma[i] + b[i] * (500 / E_c[i])**b[i]
    else:
        mev_500_index[i] = alpha[i] + 2*beta[i] * np.log(500 / E_0[i])

In [104]:
in_data = np.vstack((glat, glon, ln_energy_flux100, ln_unc_energy_flux100, ln_signif_curve, ln_var_index, hr12, hr23, hr34, hr45, mev_500_index))
out_data = np.isin(data['CLASS1'], agn_classes).astype(float)
np.savez_compressed('data/3fgl_simple_data.npz', in_data=in_data, out_data=out_data)

In [106]:
hdul_4fgl[1].data.names

[
    'Source_Name',
    'DataRelease',
    'RAJ2000',
    'DEJ2000',
    'GLON',
    'GLAT',
    'Conf_68_SemiMajor',
    'Conf_68_SemiMinor',
    'Conf_68_PosAng',
    'Conf_95_SemiMajor',
    'Conf_95_SemiMinor',
    'Conf_95_PosAng',
    'ROI_num',
    'Extended_Source_Name',
    'Signif_Avg',
    'Pivot_Energy',
    'Flux1000',
    'Unc_Flux1000',
    'Energy_Flux100',
    'Unc_Energy_Flux100',
    'SpectrumType',
    'PL_Flux_Density',
    'Unc_PL_Flux_Density',
    'PL_Index',
    'Unc_PL_Index',
    'LP_Flux_Density',
    'Unc_LP_Flux_Density',
    'LP_Index',
    'Unc_LP_Index',
    'LP_beta',
    'Unc_LP_beta',
    'LP_SigCurv',
    'PLEC_Flux_Density',
    'Unc_PLEC_Flux_Density',
    'PLEC_Index',
    'Unc_PLEC_Index',
    'PLEC_Expfactor',
    'Unc_PLEC_Expfactor',
    'PLEC_Exp_Index',
    'Unc_PLEC_Exp_Index',
    'PLEC_SigCurv',
    'Npred',
    'Flux_Band',
    'Unc_Flux_Band',
    'nuFnu_Band',
    'Sqrt_TS_Band',
    'Variability_Index',
    'Frac_Variability',
    'Unc_Frac_Variability',
    'Signif_Peak',
    'Flux_Peak',
    'Unc_Flux_Peak',
    'Time_Peak',
    'Peak_Interval',
    'Flux_History',
    'Unc_Flux_History',
    'Sqrt_TS_History',
    'ASSOC_FGL',
    'ASSOC_FHL',
    'ASSOC_GAM1',
    'ASSOC_GAM2',
    'ASSOC_GAM3',
    'TEVCAT_FLAG',
    'ASSOC_TEV',
    'CLASS1',
    'CLASS2',
    'ASSOC1',
    'ASSOC2',
    'ASSOC_PROB_BAY',
    'ASSOC_PROB_LR',
    'RA_Counterpart',
    'DEC_Counterpart',
    'Unc_Counterpart',
    'Flags'
]

In [116]:
hdul_4fgl[1].data.names

[
    'Source_Name',
    'DataRelease',
    'RAJ2000',
    'DEJ2000',
    'GLON',
    'GLAT',
    'Conf_68_SemiMajor',
    'Conf_68_SemiMinor',
    'Conf_68_PosAng',
    'Conf_95_SemiMajor',
    'Conf_95_SemiMinor',
    'Conf_95_PosAng',
    'ROI_num',
    'Extended_Source_Name',
    'Signif_Avg',
    'Pivot_Energy',
    'Flux1000',
    'Unc_Flux1000',
    'Energy_Flux100',
    'Unc_Energy_Flux100',
    'SpectrumType',
    'PL_Flux_Density',
    'Unc_PL_Flux_Density',
    'PL_Index',
    'Unc_PL_Index',
    'LP_Flux_Density',
    'Unc_LP_Flux_Density',
    'LP_Index',
    'Unc_LP_Index',
    'LP_beta',
    'Unc_LP_beta',
    'LP_SigCurv',
    'PLEC_Flux_Density',
    'Unc_PLEC_Flux_Density',
    'PLEC_Index',
    'Unc_PLEC_Index',
    'PLEC_Expfactor',
    'Unc_PLEC_Expfactor',
    'PLEC_Exp_Index',
    'Unc_PLEC_Exp_Index',
    'PLEC_SigCurv',
    'Npred',
    'Flux_Band',
    'Unc_Flux_Band',
    'nuFnu_Band',
    'Sqrt_TS_Band',
    'Variability_Index',
    'Frac_Variability',
    'Unc_Frac_Variability',
    'Signif_Peak',
    'Flux_Peak',
    'Unc_Flux_Peak',
    'Time_Peak',
    'Peak_Interval',
    'Flux_History',
    'Unc_Flux_History',
    'Sqrt_TS_History',
    'ASSOC_FGL',
    'ASSOC_FHL',
    'ASSOC_GAM1',
    'ASSOC_GAM2',
    'ASSOC_GAM3',
    'TEVCAT_FLAG',
    'ASSOC_TEV',
    'CLASS1',
    'CLASS2',
    'ASSOC1',
    'ASSOC2',
    'ASSOC_PROB_BAY',
    'ASSOC_PROB_LR',
    'RA_Counterpart',
    'DEC_Counterpart',
    'Unc_Counterpart',
    'Flags'
]